In [1]:
# Distribution ✨ jit ❇ Demo 💪 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

%load_ext autoreload
%autoreload 2
%env "WANDB_NOTEBOOK_NAME" "run.ipynb" # ❕same as notebook
# from jax.config import config
# config.update('jax_disable_jit', True)

env: "WANDB_NOTEBOOK_NAME"="run.ipynb" # ❕same as notebook


In [2]:
from functools import partial
import numpy as np
import re as regex
from pprint import pprint

import jax
from jax import pmap, grad
from jax import numpy as jnp
from jax import random as rnd
from flax.training.train_state import TrainState
from flax.core.frozen_dict import FrozenDict	
import optax

from pyfig import Pyfig
from hwat import FermiNet, sample, compute_ke_b, PotentialEnergy
from hwat import PotentialEnergy, sample, compute_ke_b
import wandb

from utils import flat_any
from utils import debug
from hwat import check_antisym, compute_pe


In [3]:
# with jax.checking_leaks():

# args = {'l_e':[4,], 'a_z':[4,], 'n_u': 2,'n_b': 16, 'n_sv': 16, 'n_pv': 8, 'corr_len': 20, 'n_step': 1000, 'log_metric_step': 1,'exp_name':'demo-final'}
args = {}

c = Pyfig(wandb_mode='disabled', args=args, get_sys_arg=False)

rng = rnd.split(rnd.PRNGKey(c.seed), c.n_device)
r = c.data.init_walker(rng, n_b=c.data.n_b)
print(f'innit: r {r.shape} rng {rng.shape} ✅')


@partial(jax.pmap, axis_name='dev', in_axes=(0,0))
def create_train_state(rng, r):
    model = c.partial(FermiNet)  
    params = model.init(rng, r)['params']
    print(params)
    return TrainState.create(apply_fn=model.apply, params=params, tx=optax.adam(c.opt.lr))
state = create_train_state(rng, r)
print('Model ✅')


compute_pe_vmap = jax.vmap(partial(compute_pe, a=c.data.a, a_z=c.data.a_z), in_axes=(0, None, None))


@partial(jax.pmap, axis_name='dev', in_axes=(0,0,0,0))
def equil(rng, state, r, deltar):
    r, v_sam = sample(rng, state, r, deltar)
    return r, v_sam
print('Equil ✅')

deltar = jnp.ones((c.n_device, 1), dtype=r.dtype)*0.02
for step in range(1, 2):
    r, v_sam = equil(rng, state, r, deltar)

# check_antisym(c, rng, r)

Path:  /home/amawi/projects/hwat/exp/demo-final/ipAYkZv ✅
System 
{'a': array([[0., 0., 0.]]),
 'a_z': array([4.]),
 'acc_target': 0.5,
 'corr_len': 20,
 'equil_len': 10000,
 'init_walker': functools.partial(<function init_walker at 0x7fe74c444280>, n_b=512, n_u=2, n_d=2, center=array([[0., 0., 0.]]), std=0.1),
 'l_e': [4],
 'n_b': 512,
 'n_d': 2,
 'n_e': 4,
 'n_u': 2,
 'with_sign': False}
Model 
{'compute_p_emb': functools.partial(<function compute_emb at 0x7fe74c416f80>, terms=['rr']),
 'compute_s_emb': functools.partial(<function compute_emb at 0x7fe74c416f80>, terms=['r', 'ra'], a=array([[0., 0., 0.]])),
 'n_det': 1,
 'n_fb': 3,
 'n_fbv': 128,
 'n_pv': 16,
 'n_sv': 32,
 'terms_p_emb': ['rr'],
 'terms_s_emb': ['r', 'ra']}


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


run:   ✅
innit: r (1, 512, 4, 3) rng (1, 2) ✅
FrozenDict({
    Dense_0: {
        bias: Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[32,2])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Dense_1: {
        bias: Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[32,2])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Dense_2: {
        bias: Traced<ShapedArray(float32[6])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[3,6])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Dense_3: {
        bias: Traced<ShapedArray(float32[6])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[3,6])>with<DynamicJaxprTrace(level=0/1)>,
    },
    p_0: {
        bias: Traced<ShapedArray(float32[16])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[4,16])>with<DynamicJaxprTrace(level=0/1)>,
    },
 

ScopeCollectionNotFound: Tried to access "kernel" from collection "params" in "/vmap()/p_0" but the collection is empty. (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ScopeCollectionNotFound)

In [10]:
# Equilibration
n_equil = 1

wandb.define_metric("*", step_metric="eq/step")
for step in range(1, 100):
    r, v_sam = equil(rng, state, r, deltar)
    rng, deltar, acc = v_sam['rng'], v_sam['deltar'], v_sam['acc']
    # if not (step % c.log_metric_step//10):
    if not (step % 1):
        wandb.log({'eq/step':step, 'acc': acc.mean(), 'deltar': deltar.mean()})
print('Walkers ✅ Training Variables ✅')


@partial(pmap, in_axes=(0, 0, 0, 0))
def train_step(rng, state, r, deltar):
    
    r_step, v_sam = sample(rng, state, r, deltar)
    
    ke = compute_ke_b(state, r_step)
    pe = compute_pe_vmap(r)
    e = pe + ke

    def loss(params):
        return ((e - jnp.mean(e)) * state.apply_fn(params, r_step)).mean()
    
    grads = grad(loss)(state.params)

    v = dict(
        rng=rng, \
        grads=grads, r=r_step, \
        pe=pe, ke=ke, e=e, \
        deltar=v_sam['deltar'], acc=v_sam['acc']
    )
                    
    return state, v

@pmap
def update_step(state, grads):
    return state.apply_gradients(grads=grads)


print('Train Step ✅')

print('Go seek: ', c.wandb_c.wandb_run_path)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/amawi/.conda/envs/dex/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4346/1095880024.py", line 6, in <module>
    r, v_sam = equil(rng, state, r, deltar)
  File "/home/amawi/.conda/envs/dex/lib/python3.10/site-packages/jax/_src/traceback_util.py", line 162, in reraise_with_filtered_traceback
    return fun(*args, **kwargs)
  File "/home/amawi/.conda/envs/dex/lib/python3.10/site-packages/jax/_src/api.py", line 2255, in cache_miss
    execute = pxla.xla_pmap_impl_lazy(fun_, *tracers, **params)
  File "/home/amawi/.conda/envs/dex/lib/python3.10/site-packages/jax/interpreters/pxla.py", line 974, in xla_pmap_impl_lazy
    compiled_fun, fingerprint = parallel_callable(
  File "/home/amawi/.conda/envs/dex/lib/python3.10/site-packages/jax/linear_util.py", line 303, in memoized_fun
    ans = call(fun, *args)
  File "/home/amawi/.co

In [ ]:
def collect_stats(k, v, new_d, p='tr', suf='', sep='/', sep_long='-'):
	depth = p.count('/')
	if depth > 1:
		sep = sep_long
	if isinstance(v, dict):
		for k_sub,v_sub in v.items():
			collect_stats(k, v_sub, new_d, p=(p+sep+k_sub))
	else:
		new_d[p+sep+k+suf] = v
	return new_d

def compute_metrix(d:dict, mode='tr'):
	pattern_ignore = ['Dense']
	fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
	)
	_d = {}
	for k,v in d.items():
		k = fancy.get(k, k)
		v = jax.device_get(v)
		if isinstance(v, FrozenDict):
			v = v.unfreeze()
		
		v_mean = jax.tree_map(lambda x: x.mean(), v) if not np.isscalar(v) else v
		v_std = jax.tree_map(lambda x: x.std(), v) if not np.isscalar(v) else 0.

		_d = collect_stats(k, v_mean, _d, p=mode, suf=r'_\mu$')
		_d = collect_stats(k, v_std, _d, p=mode, suf=r'_\sigma$')

	# return {k:v for k,v in _d.items() if not any([regex.match(k, pat) for pat in pattern_ignore])}
	return {k:v for k,v in _d.items() if not any([pat in k for pat in pattern_ignore])}


In [ ]:
wandb.define_metric("*", step_metric="tr/step")


for step in range(1, 100*c.n_step+1):
    state, data = train_step(rng, state, r, deltar)
    state = update_step(state, data['grads'])
    rng, deltar, r = data['rng'], data['deltar'], data['r']

    # if not (step % c.log_metric_step):
    if not (step % 20):
        metrix = compute_metrix(data)
        wandb.log({'tr/step':step, **metrix})
        m = ' '.join([f'{k} {v:.5f} ' for k,v in metrix.items() if 'E_' in k])
        print(f'Step {step} {m}')


    # if not (step % c.log_state_step):
    #     ...

Step 20 tr/$E_\mu$ -2.73005  tr/$E_\sigma$ 5.06943 
Step 40 tr/$E_\mu$ -1.81446  tr/$E_\sigma$ 6.39318 
Step 60 tr/$E_\mu$ -1.07435  tr/$E_\sigma$ 11.44112 
Step 80 tr/$E_\mu$ -1.72587  tr/$E_\sigma$ 8.61826 
Step 100 tr/$E_\mu$ -1.66523  tr/$E_\sigma$ 9.26621 
Step 120 tr/$E_\mu$ 0.25484  tr/$E_\sigma$ 36.94001 
Step 140 tr/$E_\mu$ -2.99419  tr/$E_\sigma$ 11.76913 
Step 160 tr/$E_\mu$ -3.09554  tr/$E_\sigma$ 10.80970 
Step 180 tr/$E_\mu$ -2.44951  tr/$E_\sigma$ 7.11648 
Step 200 tr/$E_\mu$ -2.74008  tr/$E_\sigma$ 8.61677 
Step 220 tr/$E_\mu$ -2.86974  tr/$E_\sigma$ 5.21318 
Step 240 tr/$E_\mu$ -2.15701  tr/$E_\sigma$ 16.82730 
Step 260 tr/$E_\mu$ -2.96074  tr/$E_\sigma$ 5.17586 
Step 280 tr/$E_\mu$ -2.99612  tr/$E_\sigma$ 5.82232 
Step 300 tr/$E_\mu$ -3.27481  tr/$E_\sigma$ 7.97316 
Step 320 tr/$E_\mu$ -2.53196  tr/$E_\sigma$ 5.02664 
Step 340 tr/$E_\mu$ -2.45331  tr/$E_\sigma$ 5.01992 
Step 360 tr/$E_\mu$ -3.10504  tr/$E_\sigma$ 11.54333 
Step 380 tr/$E_\mu$ -2.74311  tr/$E_\sigma$ 5

In [ ]:

# heroku create

# codey code

# 🌵 


In [ ]:
""" bone zone

# 💓 
class tr_data:
    x:jnp.ndarray=x
    rng:jnp.ndarray=rng

# Likely needed for updating outside the loop, not sure
@jax.pmap
def update_model(state, grads):
  return state.apply_gradients(grads=grads)
print('Update ✅')


# add mutable states to the trian state where the parallelism is handled? 
from flax.training import train_state

class TrainState(train_state.TrainState):
  batch_stats: flax.core.FrozenDict[str, Any]

# how to include other variables
def loss_fn(params):
    outputs, new_model_state = state.apply_fn(
        {'params': params, 'batch_stats': state.batch_stats},
        inputs,
        mutable=['batch_stats'])
    loss = xent_loss(outputs, labels)
    return loss, new_model_state

  (loss, new_model_state), grads = jax.value_and_grad(
      loss_fn, has_aux=True)(state.params)
  new_state = state.apply_gradients(
      grads=grads,
      batch_stats=new_model_state['batch_stats'],
  )

class arg_cls(Pyfig):
    def __init__(_i):
        pass
arg_cls.data.n_e = 6
arg_cls.data.n_u = 6
arg_cls.data.n_b = 6
arg_cls.n_step = 20
arg_cls.log_metric_step = 5
arg_cls.exp_name = 'demo'
args = flat_any(arg_cls().d)

"""